# Arbre enraciné de profit maximum avec des contraintes de capacité

Etant donné un ensemble de sites $V$ et une matrice de distance $D = (d_{ij})$ entre chaque paire de sites, le problème du $p$-centre consiste à sélectionner $p$ sites de $V$ afin qu'ils agissent comme des centres et à affecter les autres sites au centre disponible le plus proche tout en minimisant la plus grande distance entre un site et son centre. Des contraintes supplémentaires, telles que des contraintes de précédence ou des contraintes de capacité, peuvent apparaître dans certaines applications. Ces problèmes sont souvent résolus par des méthodes de décomposition où le problème original est décomposé en sous-problèmes consistant à construire des centres réalisables.

Le but de ce projet est de proposer des modèles mathématiques pour la résolution d'un sous-problème associé à un site donné jouant le rôle de centre.

## Présentation du problème

Soient $V = \{0, \dots, n-1\}$ un ensemble de sites et $r \in V$ un centre prédéfini. A chaque site $i \in V$, on associe deux poids $w_i^1$ et $w_i^2$, une distance $d_i$ entre le site $i$ et le centre $r$, et un profit $p_i$ (pouvant être négatif). On définit également pour chaque site $i$ un prédécesseur $\pi_i \in V$ devant être pris si $i$ est pris dans la solution. On suppose que les données satisfont la propriété que la distance au centre d'un sommet $i$ est supérieure ou égale à la distance au centre de son précédesseur $\pi_i$. L'ensemble des prédécesseurs défini un arbre enraciné en le centre $r$. Soient $C^1$ et $C^2$ deux capacités.

Le problème consiste à selectionner un ensemble de sommets qui seront affectés au centre $r$, dont le poids total (centre inclu), pour chacun des poids $w^1$ et $w^2$ est inférieur aux capacités $C^1$ et $C^2$, et tel que si un sommet $i$ est sélectionné, alors sont prédécesseur $\pi_i$ est également sélectionné.

L'objectif du problème est de maximiser le profit total des sites sélectionnés (en incluant le centre $r$) auquel on soustrait la distance au centre (également appelée rayon) du sommet sélectionné le plus éloigné.


## Données

Le répertoire `instances` est un ensemble de jeux de données pour ce problème. Chaque fichier correspond à un jeu de données et contient les informations suivantes (la numérotation des sites commence à 0) :

- Première ligne : Nombre de sites, $C^1$, $C^2$, numéro du centre.
- Une ligne pour chaque site indiquant : numéro du prédécesseur $\pi_i$, distance $d_i$ du site au centre, le profit $p_i$ du site, le poids $w_i^1$, le poids $w_i^2$.

Si pour un site $i$, son prédécesseur vaut $\pi_i = -1$, soit $i$ est le centre $r$ et il doit être pris dans la solution, soit le site $i$ n'est pas le centre et auquel cas, le site ne peut pas être pris dans la solution.

Implémentons ici la lecture des données :

In [1]:
def parser(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        n, c1, c2, center = lines[0].split()
        pred = []
        dist = []
        profit = []
        w1 = []
        w2 = []
        for line in lines[1:-3]:
            pred.append(int(line.split()[0]))
            dist.append(float(line.split()[1]))
            profit.append(float(line.split()[2]))
            w1.append(float(line.split()[3]))
            w2.append(float(line.split()[4]))
        return int(n), float(c1), float(c2), int(center), pred, dist, profit, w1, w2

## Premier modèle

Considérons les variables :

- $x_i = \left\{ \begin{array}{l}
1 \text{ si le site } i \in V \text{ est sélectionné,}\\
0 \text{ sinon.}
\end{array}\right.$
- $R \in \mathbb{R} $ la valeur du rayon (la distance au centre du sommet sélectionné le plus éloigné).

Implémentons ce premier modèle et testons le sur une instance :

In [2]:
data = "./Instances/btk_031_0_0.dat"

In [3]:
n, c1, c2, center, pred, dist, profit, w1, w2 = parser(data)
#print(n, c1, c2, center, pred, dist, profit, w1, w2)

In [4]:
from mip import *

model_1 = Model(name='AEPMCC_cf', solver_name="CBC")

x = [model_1.add_var(name="x_%s" % i, var_type=BINARY) for i in range(n+1)]
r = model_1.add_var(name="r", var_type=CONTINUOUS)

model_1.objective = maximize(xsum(profit[i] * x[i] for i in range(n)) - r)

model_1.add_constr(xsum(w1[i] * x[i] for i in range(n)) <= c1, name="c1")
model_1.add_constr(xsum(w2[i] * x[i] for i in range(n)) <= c2, name="c2")

for i in range(n):
    if i != center:
        model_1.add_constr(x[i] <= x[pred[i]], name="c3_%s" % i)

for i in range(n):
    model_1.add_constr(x[i]*dist[i] <= r, name="c4_%s" % i)

model_1.add_constr(x[center] == 1, name="c5")

model_1.add_constr(x[-1] == 0, name="c6")

model_1.optimize()
model_1.objective_value

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 56 (-9) rows, 30 (-3) columns and 152 (-18) elements
Clp1000I sum of infeasibilities 0.000401253 - average 7.16523e-06, 10 fixed columns
Coin0506I Presolve 36 (-20) rows, 20 (-10) columns and 89 (-63) elements
Clp0029I End of values pass after 20 iterations
Clp0000I Optimal - objective value 11176.138
Clp0000I Optimal - objective value 11176.138
Coin0511I After Postsolve, objective 11176.138, infeasibilities - dual 0 (0), primal 0 (0)
Clp0014I Perturbing problem by 0.001% of 0.3879757 - largest nonzero change 0 ( 0%) - largest zero change 2.5061934e-05
Clp0000I Optimal - objective value 11176.138
Clp0000I Optimal - objective value 11176.138
Clp0000I Optimal - objective value 11176.138
Coin0511I After Postsolve, objective 11176.138, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 11176.1378 

6914.7502110000005

## Deuxième modèle

Il est possible de modéliser le rayon de manière plus précise. Pour cela, on définit la fonction $\sigma : V \mapsto \{1, \dots, n\}$ qui associe à chaque site $i \in V$ sa position dans l'ordre croissant des distances des sites au centre et $\sigma^{-1}(k)$ la fonction inverse qui renvoie le site en position $k$ dans cet ordre\
($0 = d_r = d_{\sigma^{-1}(1)} \leq d_{\sigma^{-1}(2)} \leq \dots \leq d_{\sigma^{-1}(n)}$). Pour tout site $i \in V \setminus \{r\}$ différent du centre $r$, on pose $\Delta_i = d_i - d_{\sigma^{-1}(\sigma(i)-1)}$ la différence de distance entre le site $i$ et son prédécesseur dans l'ordre défini pas $\sigma$.

Considérons les variables suivantes :

- $x_i = \left\{ \begin{array}{l}
1 \text{ si le site } i \in V \text{ est sélectionné,}\\
0 \text{ sinon.}
\end{array}\right.$      
- $y_i = \left\{ \begin{array}{l}
1 \text{ si le site } i \in V \text{ a une distance plus petite ou égale que le rayon du centre,}\\
0 \text{ sinon.}
\end{array}\right.$

Notons qu'un sommet peut avoir une distance plus petite que le rayon, sans nécessairement être sélectionné dans la solution. En revanche, un sommet sélectionné doit forcement avoir une distance plus petite ou égale au rayon. Notons également que le rayon d'un centre est la somme des $\Delta_i$ sur tous les sites $i$ ayant une distance plus petite ou égale au rayon.

Implémentons ce second modèle ci-dessous et testons le sur la même instance :

In [5]:
n, c1, c2, center, pred, dist, profit, w1, w2 = parser(data)
#print(n, c1, c2, center, pred, dist, profit, w1, w2)

In [6]:
x_sorted = [i for i in range(n)]
dist_sorted = dist.copy()

for i in range(n):
    min = i
    for j in range(i, n):
        if(dist_sorted[j] < dist_sorted[min]):
            min = j
    if(min != i):
        temp = dist_sorted[i]
        dist_sorted[i] = dist_sorted[min]
        dist_sorted[min] = temp
        temp = x_sorted[i]
        x_sorted[i] = x_sorted[min]
        x_sorted[min] = temp

def sigma(i):
    for k in range(n):
        if(x_sorted[k] == i):
            return k

def sigma_inv(k):
    return x_sorted[k]

def delta(i):
    if i == center:
        return 0
    return dist[i] - dist[sigma_inv(sigma(i)-1)]

model_2 = Model(name='AEPMCC_cf', solver_name="CBC")

x = [model_2.add_var(name="x_%s" % i, var_type=BINARY) for i in range(n+1)]
y = [model_2.add_var(name="y_%s" % i, var_type=BINARY) for i in range(n)]

model_2.objective = maximize(xsum(profit[i] * x[i] - y[i] * delta(i) for i in range(n)))

model_2.add_constr(xsum(w1[i] * x[i] for i in range(n)) <= c1, name="c1")
model_2.add_constr(xsum(w2[i] * x[i] for i in range(n)) <= c2, name="c2")

for i in range(n):
    if i != center:
        model_2.add_constr(x[i] <= x[pred[i]], name="c3_%s" % i)

model_2.add_constr(x[center] == 1, name="c4")

for i in range(n):
    model_2.add_constr(xsum(x[sigma_inv(k)] for k in range(i, n)) <= y[sigma_inv(i)]*(n-i), name="c5_%s" % i)

model_2.add_constr(x[-1] == 0, name="c6")

model_2.optimize()
model_2.objective_value

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 45 (-20) rows, 47 (-16) columns and 359 (-277) elements
Clp1000I sum of infeasibilities 6.4559e-10 - average 1.43465e-11, 42 fixed columns
Coin0506I Presolve 5 (-40) rows, 5 (-42) columns and 13 (-346) elements
Clp0029I End of values pass after 5 iterations
Clp0000I Optimal - objective value 2959.9315
Clp0000I Optimal - objective value 2959.9315
Coin0511I After Postsolve, objective 2959.9315, infeasibilities - dual 0 (0), primal 0 (0)
Clp0014I Perturbing problem by 0.001% of 4.0694836 - largest nonzero change 0 ( 0%) - largest zero change 2.9394864e-05
Clp0000I Optimal - objective value 12539.039
Clp0000I Optimal - objective value 12539.039
Clp0000I Optimal - objective value 12539.039
Coin0511I After Postsolve, objective 12539.039, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 12539.03879 - 0 iterations time 0.012, Presolve 0.00, Idiot 0.01

Starting MIP optim

6914.7502110000005